# OS Status: conventional NN models

In [ ]:
#Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
#Importing modules for data preprocessing, model training
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
#Importing metrics for model evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
#Importing modules for building the neural network
from tensorflow.keras import layers, regularizers
#For model interpretation
import shap


# for reproducibility, the value is set for conventional reasons
SEED = 42
tf.keras.utils.set_random_seed(SEED)



In [ ]:
# load the data
data = pd.read_csv('dataset_b', encoding='latin-1', sep=',') # request the dataset to the author
#data.head()

In [3]:
# target column : "os_status", binary variable
# relevant columns
relevant_columns = ['age', 'dcr', 'dnlr', 'histology', 'immuno_line', 'iorr', 
                    'ldhpre', 'leucotpre', 'nb_meta_beforeimmuno', 'neuttpre', 
                     'ps_befimmuno', 'sex', 'smoking_history', 'os_status']

data = data[relevant_columns]
data = data.dropna(axis=0)
data['dcr'] = data['dcr'].astype(int)
data['age'] = data['age'].astype(int)
data['iorr'] = data['iorr'].astype(int)
data['ps_befimmuno'] = data['ps_befimmuno'].astype(int)

#data.head()

In [4]:
data['histology'] = data['histology'].str.lower()
data['sex'] = data['sex'].str.lower()
data['smoking_history'] = data['smoking_history'].str.lower()

In [5]:
#to randomize the data
data = data.sample(frac=1, random_state=SEED)

# one-hot encoding
one_hot_data = pd.get_dummies(data, columns=['histology', 'sex', 'smoking_history'])

one_hot_data = one_hot_data.rename(columns={
    'histology_Adenocarcinoma': 'histology_adenocarcinoma',
    'histology_Squamous': 'histology_squamous',
    'histology_Nsclc_other': 'histology_nsclc_other',
    'histology_Large_cells': 'histology_large_cells',
    'sex_Male': 'sex_male',
    'sex_Female': 'sex_female',
    'smoking_history_Non_smoker': 'smoking_history_non_smoker',
    'smoking_history_Former': 'smoking_history_former',
    'smoking_history_Current': 'smoking_history_current',
    'smoking_history_Unk': 'smoking_history_unk'
})

#one_hot_data.head()

In [ ]:
# replace boolean values with 0 and 1
for col in ['histology_adenocarcinoma','histology_squamous','histology_nsclc other',
    'histology_large cells','sex_male','sex_female','smoking_history_non smoker','smoking_history_former','smoking_history_current',
     'smoking_history_unk']:
    one_hot_data[col] = one_hot_data[col].replace({False: 0, True: 1})

In [ ]:
# split the data into features and target
X = one_hot_data[one_hot_data.columns.difference(['os_status'])]
y = data['os_status']


# First split: training+validation vs test (80% vs 20%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y  # stratify to maintain class distribution
)

# Second split: training vs validation (75% vs 25% del 80%)
# This results in 60% train, 20% val, 20% test
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp
)

In [ ]:
#This ensures that all numerical features contribute equally
numerical_features = ['age', 'dcr', 'dnlr', 'ldhpre', 'leucotpre', 
                      'nb_meta_beforeimmuno', 'neuttpre', 'ps_befimmuno']
scaler = StandardScaler()

binary_features = [col for col in X.columns if col not in numerical_features]

scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_val_scaled = X_val.copy() 
X_test_scaled = X_test.copy()
X_scaled = X.copy()


X_scaled[numerical_features] = scaler.fit_transform(X_scaled[numerical_features])
X_train_scaled[numerical_features] = scaler.fit_transform(X_train_scaled[numerical_features])
X_val_scaled[numerical_features] = scaler.transform(X_val_scaled[numerical_features])
X_test_scaled[numerical_features] = scaler.transform(X_test_scaled[numerical_features])


In [ ]:
# model with 1 hidden layer with 512 neurons and dropout 0.4
mlp_model = tf.keras.models.Sequential([
    layers.Dense(512, activation='relu', input_shape=(X.shape[1],)),
    layers.Dropout(0.4),
    layers.Dense(1, activation='sigmoid')
])


# Compile the model
mlp_model.compile(optimizer='adam',
                  loss='binary_crossentropy',  
                  metrics=['accuracy'])

# Train the model
history = mlp_model.fit(X_train_scaled, y_train,
                         validation_data=(X_val_scaled, y_val),
                         epochs=500, batch_size=16)

epochs = range(1, len(history.history['loss']) + 1)

# Plot of the loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['loss'], 'b', label='Training Loss')
plt.plot(epochs, history.history['val_loss'], 'r', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot of accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['accuracy'], 'b', label='Training Accuracy')
plt.plot(epochs, history.history['val_accuracy'], 'r', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation accuracy')
plt.legend()

plt.show()

In [10]:
# Predict class probabilities
prob_predictions = mlp_model.predict(X_test_scaled)

# Get the predicted class index
prob_predictions = np.squeeze(prob_predictions)

# Convert the probabilities into binary predictions
class_predictions = (prob_predictions >= 0.5).astype(int)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


In [11]:
X_scaled = X.copy()
X_scaled[numerical_features] = scaler.fit_transform(X_scaled[numerical_features])

In [ ]:
accuracy = accuracy_score(y_test, class_predictions)
precision = precision_score(y_test, class_predictions)
recall = recall_score(y_test, class_predictions)
f1 = f1_score(y_test, class_predictions)
roc_auc = roc_auc_score(y_test, prob_predictions)  

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"AUC-ROC: {roc_auc:.4f}")

# Confusion Matrix
cm = confusion_matrix(y_test, class_predictions)
print("Confusion Matrix:")
print(cm)

prob_predictions = mlp_model.predict(X_scaled)
prob_predictions = np.squeeze(prob_predictions)
class_predictions = (prob_predictions >= 0.5).astype(int)
data['Predicted'] = class_predictions

print(data.tail(10))

In [13]:
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [ ]:
background = shap.kmeans(X_train_scaled, 10)
explainer = shap.KernelExplainer(lambda x: mlp_model.predict(x), background)

X_subset = X_test_scaled_df.sample(50, random_state=42)
shap_values = explainer.shap_values(X_subset, nsamples=50, silent=True)

if isinstance(shap_values, list):
    shap_values = shap_values[0]
if shap_values.ndim == 3:
    shap_values = shap_values[:, :, 0]

print(f"SHAP values shape: {shap_values.shape}, X_subset shape: {X_subset.shape}")

plt.figure(figsize=(10, 8))
shap.summary_plot(
    shap_values,
    features=X_subset,
    feature_names=X_subset.columns,
    plot_type='dot',
    max_display=len(binary_features + numerical_features),
    show=False
)
plt.title("SHAP Summary Plot – Class 1")
plt.tight_layout()
plt.savefig("shap_summary_binary.png")
plt.show()


In [ ]:
# model with 1 hidden layer with 512 neurons, L2 regularization 0.01 and increased dropout 0.4
mlp_model = tf.keras.models.Sequential([
    layers.Dense(512, activation='relu', input_shape=(X.shape[1],), kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.4),
    layers.Dense(1, activation='sigmoid')
])


# Compile the model
mlp_model.compile(optimizer='adam',
                  loss='binary_crossentropy',  
                  metrics=['accuracy'])

# Train the model
history = mlp_model.fit(X_train_scaled, y_train,
                         validation_data=(X_val_scaled, y_val),
                         epochs=500, batch_size=16)

epochs = range(1, len(history.history['loss']) + 1)

# Plot of the loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['loss'], 'b', label='Training Loss')
plt.plot(epochs, history.history['val_loss'], 'r', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot of accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['accuracy'], 'b', label='Training Accuracy')
plt.plot(epochs, history.history['val_accuracy'], 'r', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation accuracy')
plt.legend()

plt.show()

In [35]:
# Predict class probabilities
prob_predictions = mlp_model.predict(X_test_scaled)

# Get the predicted class index
prob_predictions = np.squeeze(prob_predictions)

# Convert the probabilities into binary predictions
class_predictions = (prob_predictions >= 0.5).astype(int)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


In [ ]:
accuracy = accuracy_score(y_test, class_predictions)
precision = precision_score(y_test, class_predictions)
recall = recall_score(y_test, class_predictions)
f1 = f1_score(y_test, class_predictions)
roc_auc = roc_auc_score(y_test, prob_predictions)  

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"AUC-ROC: {roc_auc:.4f}")

# Confusion Matrix
cm = confusion_matrix(y_test, class_predictions)
print("Confusion Matrix:")
print(cm)

prob_predictions = mlp_model.predict(X_scaled)
prob_predictions = np.squeeze(prob_predictions)
class_predictions = (prob_predictions >= 0.5).astype(int)
data['Predicted'] = class_predictions

print(data.tail(10))

In [ ]:
background = shap.kmeans(X_train_scaled, 10)
explainer = shap.KernelExplainer(lambda x: mlp_model.predict(x), background)

X_subset = X_test_scaled_df.sample(50, random_state=42)
shap_values = explainer.shap_values(X_subset, nsamples=50, silent=True)

if isinstance(shap_values, list):
    shap_values = shap_values[0]
if shap_values.ndim == 3:
    shap_values = shap_values[:, :, 0]

print(f"SHAP values shape: {shap_values.shape}, X_subset shape: {X_subset.shape}")

plt.figure(figsize=(10, 8))
shap.summary_plot(
    shap_values,
    features=X_subset,
    feature_names=X_subset.columns,
    plot_type='dot',
    max_display=len(binary_features + numerical_features),
    show=False
)
plt.title("SHAP Summary Plot – Class 1")
plt.tight_layout()
plt.savefig("shap_summary_binary.png")
plt.show()


In [ ]:
# model with 1 hidden layer with 512 neurons, Norm Layer, L2 regularization 0.01 and dropout 0.4
mlp_model = tf.keras.models.Sequential([
    layers.Dense(512, activation='relu', input_shape=(X.shape[1],), kernel_regularizer=regularizers.l2(0.01)),
    layers.LayerNormalization(),
    layers.Dropout(0.4),
    layers.Dense(1, activation='sigmoid')
])


# Compile the model
mlp_model.compile(optimizer='adam',
                  loss='binary_crossentropy',  
                  metrics=['accuracy'])

# Train the model
history = mlp_model.fit(X_train_scaled, y_train,
                         validation_data=(X_val_scaled, y_val),
                         epochs=500, batch_size=16)

epochs = range(1, len(history.history['loss']) + 1)

# Plot of the loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['loss'], 'b', label='Training Loss')
plt.plot(epochs, history.history['val_loss'], 'r', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot of accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['accuracy'], 'b', label='Training Accuracy')
plt.plot(epochs, history.history['val_accuracy'], 'r', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation accuracy')
plt.legend()

plt.show()

In [39]:
# Predict class probabilities
prob_predictions = mlp_model.predict(X_test_scaled)

# Get the predicted class index
prob_predictions = np.squeeze(prob_predictions)

# Convert the probabilities into binary predictions
class_predictions = (prob_predictions >= 0.5).astype(int)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


In [ ]:
accuracy = accuracy_score(y_test, class_predictions)
precision = precision_score(y_test, class_predictions)
recall = recall_score(y_test, class_predictions)
f1 = f1_score(y_test, class_predictions)
roc_auc = roc_auc_score(y_test, prob_predictions)  

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"AUC-ROC: {roc_auc:.4f}")

# Confusion Matrix
cm = confusion_matrix(y_test, class_predictions)
print("Confusion Matrix:")
print(cm)

prob_predictions = mlp_model.predict(X_scaled)
prob_predictions = np.squeeze(prob_predictions)
class_predictions = (prob_predictions >= 0.5).astype(int)
data['Predicted'] = class_predictions

print(data.tail(10))

In [ ]:
background = shap.kmeans(X_train_scaled, 10)
explainer = shap.KernelExplainer(lambda x: mlp_model.predict(x), background)

X_subset = X_test_scaled_df.sample(50, random_state=42)
shap_values = explainer.shap_values(X_subset, nsamples=50, silent=True)

if isinstance(shap_values, list):
    shap_values = shap_values[0]
if shap_values.ndim == 3:
    shap_values = shap_values[:, :, 0]

print(f"SHAP values shape: {shap_values.shape}, X_subset shape: {X_subset.shape}")

plt.figure(figsize=(10, 8))
shap.summary_plot(
    shap_values,
    features=X_subset,
    feature_names=X_subset.columns,
    plot_type='dot',
    max_display=len(binary_features + numerical_features),
    show=False
)
plt.title("SHAP Summary Plot – Class 1")
plt.tight_layout()
plt.savefig("shap_summary_binary.png")
plt.show()


In [ ]:
# model with 1 hidden layer with 512 neurons, Batch layer, L2 regularization 0.01 and dropout 0.4
mlp_model = tf.keras.models.Sequential([
    layers.Dense(512, activation='relu', input_shape=(X.shape[1],), kernel_regularizer=regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(1, activation='sigmoid')
])



# compile the model
mlp_model.compile(optimizer='adam',
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])

# train the model
history = mlp_model.fit(X_train_scaled, y_train,
              validation_data=(X_val_scaled, y_val),
              epochs=500, batch_size=16)


epochs = range(1, len(history.history['loss']) + 1)

# Plot of the loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['loss'], 'b', label='Training Loss')
plt.plot(epochs, history.history['val_loss'], 'r', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot of accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['accuracy'], 'b', label='Training Accuracy')
plt.plot(epochs, history.history['val_accuracy'], 'r', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation accuracy')
plt.legend()

plt.show()

In [43]:
# Predict class probabilities
prob_predictions = mlp_model.predict(X_test_scaled)

# Get the predicted class index
prob_predictions = np.squeeze(prob_predictions)

# Convert the probabilities into binary predictions
class_predictions = (prob_predictions >= 0.5).astype(int)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


In [ ]:
accuracy = accuracy_score(y_test, class_predictions)
precision = precision_score(y_test, class_predictions)
recall = recall_score(y_test, class_predictions)
f1 = f1_score(y_test, class_predictions)
roc_auc = roc_auc_score(y_test, prob_predictions)  

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"AUC-ROC: {roc_auc:.4f}")

# Confusion Matrix
cm = confusion_matrix(y_test, class_predictions)
print("Confusion Matrix:")
print(cm)

prob_predictions = mlp_model.predict(X_scaled)
prob_predictions = np.squeeze(prob_predictions)
class_predictions = (prob_predictions >= 0.5).astype(int)
data['Predicted'] = class_predictions

print(data.tail(10))

In [ ]:
background = shap.kmeans(X_train_scaled, 10)
explainer = shap.KernelExplainer(lambda x: mlp_model.predict(x), background)

X_subset = X_test_scaled_df.sample(50, random_state=42)
shap_values = explainer.shap_values(X_subset, nsamples=50, silent=True)

if isinstance(shap_values, list):
    shap_values = shap_values[0]
if shap_values.ndim == 3:
    shap_values = shap_values[:, :, 0]

print(f"SHAP values shape: {shap_values.shape}, X_subset shape: {X_subset.shape}")

plt.figure(figsize=(10, 8))
shap.summary_plot(
    shap_values,
    features=X_subset,
    feature_names=X_subset.columns,
    plot_type='dot',
    max_display=len(binary_features + numerical_features),
    show=False
)
plt.title("SHAP Summary Plot – Class 1")
plt.tight_layout()
plt.savefig("shap_summary_binary.png")
plt.show()


In [ ]:
# model with 2 hidden layers, each of them with 512 neurons, L2 regularization 0.01 and dropout 0.4
mlp_model = tf.keras.models.Sequential([
    layers.Dense(512, activation='relu', input_shape=(X.shape[1],), kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.4),
    layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.4),
    layers.Dense(1, activation='sigmoid')
])


# Compile the model
mlp_model.compile(optimizer='adam',
                  loss='binary_crossentropy',  
                  metrics=['accuracy'])

# Train the model
history = mlp_model.fit(X_train_scaled, y_train,
                         validation_data=(X_val_scaled, y_val),
                         epochs=500, batch_size=16)

epochs = range(1, len(history.history['loss']) + 1)

# Plot of the loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['loss'], 'b', label='Training Loss')
plt.plot(epochs, history.history['val_loss'], 'r', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot of accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['accuracy'], 'b', label='Training Accuracy')
plt.plot(epochs, history.history['val_accuracy'], 'r', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation accuracy')
plt.legend()

plt.show()

In [47]:
# Predict class probabilities
prob_predictions = mlp_model.predict(X_test_scaled)

# Get the predicted class index
prob_predictions = np.squeeze(prob_predictions)

# Convert the probabilities into binary predictions
class_predictions = (prob_predictions >= 0.5).astype(int)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


In [ ]:
accuracy = accuracy_score(y_test, class_predictions)
precision = precision_score(y_test, class_predictions)
recall = recall_score(y_test, class_predictions)
f1 = f1_score(y_test, class_predictions)
roc_auc = roc_auc_score(y_test, prob_predictions)  

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"AUC-ROC: {roc_auc:.4f}")

# Confusion Matrix
cm = confusion_matrix(y_test, class_predictions)
print("Confusion Matrix:")
print(cm)

prob_predictions = mlp_model.predict(X_scaled)
prob_predictions = np.squeeze(prob_predictions)
class_predictions = (prob_predictions >= 0.5).astype(int)
data['Predicted'] = class_predictions

print(data.tail(10))

In [ ]:
background = shap.kmeans(X_train_scaled, 10)
explainer = shap.KernelExplainer(lambda x: mlp_model.predict(x), background)

X_subset = X_test_scaled_df.sample(50, random_state=42)
shap_values = explainer.shap_values(X_subset, nsamples=50, silent=True)

if isinstance(shap_values, list):
    shap_values = shap_values[0]
if shap_values.ndim == 3:
    shap_values = shap_values[:, :, 0]

print(f"SHAP values shape: {shap_values.shape}, X_subset shape: {X_subset.shape}")

plt.figure(figsize=(10, 8))
shap.summary_plot(
    shap_values,
    features=X_subset,
    feature_names=X_subset.columns,
    plot_type='dot',
    max_display=len(binary_features + numerical_features),
    show=False
)
plt.title("SHAP Summary Plot – Class 1")
plt.tight_layout()
plt.savefig("shap_summary_binary.png")
plt.show()


In [ ]:
# model with 2 hidden layers, each of them with 512 neurons,Norm layer, L2 regularization 0.01 and dropout 0.4
mlp_model = tf.keras.models.Sequential([
    layers.Dense(512, activation='relu', input_shape=(X.shape[1],), kernel_regularizer=regularizers.l2(0.01)), 
    layers.LayerNormalization(center=True, scale=True),
    layers.Dropout(0.4),  
    layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.4),
    layers.Dense(1, activation='sigmoid')
])


# Compile the model
mlp_model.compile(optimizer='adam',
                  loss='binary_crossentropy',  
                  metrics=['accuracy'])

# Train the model
history = mlp_model.fit(X_train_scaled, y_train,
                         validation_data=(X_val_scaled, y_val),
                         epochs=500, batch_size=16)

epochs = range(1, len(history.history['loss']) + 1)

# Plot of the loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['loss'], 'b', label='Training Loss')
plt.plot(epochs, history.history['val_loss'], 'r', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot of accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['accuracy'], 'b', label='Training Accuracy')
plt.plot(epochs, history.history['val_accuracy'], 'r', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation accuracy')
plt.legend()

plt.show()

In [51]:
# Predict class probabilities
prob_predictions = mlp_model.predict(X_test_scaled)

# Get the predicted class index
prob_predictions = np.squeeze(prob_predictions)

# Convert the probabilities into binary predictions
class_predictions = (prob_predictions >= 0.5).astype(int)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


In [ ]:
accuracy = accuracy_score(y_test, class_predictions)
precision = precision_score(y_test, class_predictions)
recall = recall_score(y_test, class_predictions)
f1 = f1_score(y_test, class_predictions)
roc_auc = roc_auc_score(y_test, prob_predictions)  

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"AUC-ROC: {roc_auc:.4f}")

# Confusion Matrix
cm = confusion_matrix(y_test, class_predictions)
print("Confusion Matrix:")
print(cm)

prob_predictions = mlp_model.predict(X_scaled)
prob_predictions = np.squeeze(prob_predictions)
class_predictions = (prob_predictions >= 0.5).astype(int)
data['Predicted'] = class_predictions

print(data.tail(10))

In [ ]:
background = shap.kmeans(X_train_scaled, 10)
explainer = shap.KernelExplainer(lambda x: mlp_model.predict(x), background)

X_subset = X_test_scaled_df.sample(50, random_state=42)
shap_values = explainer.shap_values(X_subset, nsamples=50, silent=True)

if isinstance(shap_values, list):
    shap_values = shap_values[0]
if shap_values.ndim == 3:
    shap_values = shap_values[:, :, 0]

print(f"SHAP values shape: {shap_values.shape}, X_subset shape: {X_subset.shape}")

plt.figure(figsize=(10, 8))
shap.summary_plot(
    shap_values,
    features=X_subset,
    feature_names=X_subset.columns,
    plot_type='dot',
    max_display=len(binary_features + numerical_features),
    show=False
)
plt.title("SHAP Summary Plot – Class 1")
plt.tight_layout()
plt.savefig("shap_summary_binary.png")
plt.show()


In [ ]:
# model with 2 hidden layers, each of them with 512 neurons, Norm layer, ElasticNet and  dropout 0.2
mlp_model = tf.keras.models.Sequential([
    layers.Dense(512, activation='relu', input_shape=(X.shape[1],), kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.01)), 
    layers.LayerNormalization(center=True, scale=True),
    layers.Dropout(0.2),  
    layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.01)),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')
])


# Compile the model
mlp_model.compile(optimizer='adam',
                  loss='binary_crossentropy',  
                  metrics=['accuracy'])

# Train the model
history = mlp_model.fit(X_train_scaled, y_train,
                         validation_data=(X_val_scaled, y_val),
                         epochs=500, batch_size=16)

epochs = range(1, len(history.history['loss']) + 1)

# Plot of the loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['loss'], 'b', label='Training Loss')
plt.plot(epochs, history.history['val_loss'], 'r', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot of accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['accuracy'], 'b', label='Training Accuracy')
plt.plot(epochs, history.history['val_accuracy'], 'r', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation accuracy')
plt.legend()

plt.show()

In [55]:
# Predict class probabilities
prob_predictions = mlp_model.predict(X_test_scaled)

# Get the predicted class index
prob_predictions = np.squeeze(prob_predictions)

# Convert the probabilities into binary predictions
class_predictions = (prob_predictions >= 0.5).astype(int)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


In [ ]:
accuracy = accuracy_score(y_test, class_predictions)
precision = precision_score(y_test, class_predictions)
recall = recall_score(y_test, class_predictions)
f1 = f1_score(y_test, class_predictions)
roc_auc = roc_auc_score(y_test, prob_predictions)  

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"AUC-ROC: {roc_auc:.4f}")

# Confusion Matrix
cm = confusion_matrix(y_test, class_predictions)
print("Confusion Matrix:")
print(cm)

prob_predictions = mlp_model.predict(X_scaled)
prob_predictions = np.squeeze(prob_predictions)
class_predictions = (prob_predictions >= 0.5).astype(int)
data['Predicted'] = class_predictions

print(data.tail(10))

In [ ]:
background = shap.kmeans(X_train_scaled, 10)
explainer = shap.KernelExplainer(lambda x: mlp_model.predict(x), background)

X_subset = X_test_scaled_df.sample(50, random_state=42)
shap_values = explainer.shap_values(X_subset, nsamples=50, silent=True)

if isinstance(shap_values, list):
    shap_values = shap_values[0]
if shap_values.ndim == 3:
    shap_values = shap_values[:, :, 0]

print(f"SHAP values shape: {shap_values.shape}, X_subset shape: {X_subset.shape}")

plt.figure(figsize=(10, 8))
shap.summary_plot(
    shap_values,
    features=X_subset,
    feature_names=X_subset.columns,
    plot_type='dot',
    max_display=len(binary_features + numerical_features),
    show=False
)
plt.title("SHAP Summary Plot – Class 1")
plt.tight_layout()
plt.savefig("shap_summary_binary.png")
plt.show()
